In [38]:
import json
import pyaudio
import time
from vosk import Model, KaldiRecognizer

# Load VOSK model
model_path = r"D:\vosk-model-small-en-us-0.15"
model = Model(model_path)

# Audio configuration
CHUNK = 1024  # Number of frames per buffer
FORMAT = pyaudio.paInt16  # Audio format
CHANNELS = 1
RATE = 16000  # Sampling rate

# Keyword detection and timing logic
REQUIRED_REPETITIONS = 3
TIME_THRESHOLD = 5  # Seconds
consecutive_help_count = 0
last_detection_time = 0

# Function to trigger alert
def trigger_alert():
    print("ALERT: 'HELP' detected three times consecutively!")

# Initialize and start the microphone
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
stream.start_stream()

# Initialize recognizer
recognizer = KaldiRecognizer(model, RATE)

print("Listening for 'HELP'...")

try:
    while True:
        try:
            data = stream.read(CHUNK, exception_on_overflow=False)
        except IOError:
            continue  # Skip this iteration in case of buffer overflow errors

        # Check if there is audio input
        if recognizer.AcceptWaveform(data):
            # Parse the recognized text
            result = recognizer.Result()
            text = json.loads(result).get("text", "")
            print(f"Recognized Text: {text}")  # Debugging output to check recognition

            # Split the text into words and check for each "help"
            words = text.lower().split()
            current_time = time.time()

            for word in words:
                if word == "help":
                    # Check if within time threshold to increment count
                    if current_time - last_detection_time <= TIME_THRESHOLD:
                        consecutive_help_count += 1
                    else:
                        consecutive_help_count = 1  # Reset if outside time window

                    last_detection_time = current_time  # Update detection time

                    # Trigger alert if the keyword is detected 3 times consecutively within the time frame
                    if consecutive_help_count == REQUIRED_REPETITIONS:
                        trigger_alert()
                        consecutive_help_count = 0  # Reset after alert
                else:
                    consecutive_help_count = 0  # Reset on non-help words

except KeyboardInterrupt:
    print("\nStopping...")

finally:
    stream.stop_stream()
    stream.close()
    p.terminate()

Listening for 'HELP'...

Stopping...
